In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

In [19]:
# Ignoring the warning related to Setting with copy.
pd.options.mode.chained_assignment = None

start_year = 2005

# Extract the current year
URL = 'https://www.resultados-futbol.com/ligue_1'
r = requests.get(URL)
page = r.content
soup = BeautifulSoup(page, 'html5lib')

current_year = int(soup.find('div', class_ = "titular-data").text.strip()[0:4])+1

# Define the countries and the years
Years = range(start_year,current_year+1)

Countries = ['Portugal','Spain','England','Italy','Germany','France']

# Create table_fifa_all_years

In [20]:
%%time

# creation of an empty data frame for all countries, all years
table_fifa_all_years = pd.DataFrame()

for Year in Years:

    year = str(Year)[-2:]

    # creation of an empty data frame for all countries, one year
    table_fifa_all_countries = pd.DataFrame()
    
    # Each country league has a corresponding code in the Fifa index website
    for Country in Countries:
        if Country == 'Portugal':
            league = 308
        elif Country == 'Spain':
            league = 53
        elif Country == 'England':
            league = 13
        elif Country == 'Italy':
            league = 31
        elif Country == 'Germany':
            league = 19
        elif Country == 'France':
            league = 16
        else:
            print('Country not available, please choose between Portugal,\
        Spain, England, Italy, Germany or France')


        # Per each country

        URL = f'https://www.fifaindex.com/teams/fifa{year}/?league={league}&order=desc'

        # Ratings
        webpage = pd.read_html(URL)
        table_fifa_country = pd.DataFrame(webpage[0])
        table_fifa_country.dropna(subset = ["Name"], inplace=True)
        table_fifa_country.drop(table_fifa_country.columns.difference(
            ['Name','ATT','MID','DEF','OVR']), 1, inplace=True)
        table_fifa_country.insert(0, "Year", Year, True)
        table_fifa_country.insert(1, "Country", Country, True)
        table_fifa_country.reset_index(drop=True,inplace=True)

        # Rivals Teams & Budget
        
        # Each team has an ID code in its correspondant URL in Fifa index
        # Let's extract it so we can use it to get other info
        r = requests.get(URL)
        page = r.content
        soup = BeautifulSoup(page, 'html5lib')

        teams_URL_code_in_national_league = []

        for a in soup.find_all('a', class_ = "link-team"):
            if a['href'] not in teams_URL_code_in_national_league:
                teams_URL_code_in_national_league.append(a['href'])

        table_fifa_country["Link-team"] = teams_URL_code_in_national_league


        Rival_team_list = []
        Team_budget_list = []
        #Team_budget_list_aux = []

        for link_team in list(table_fifa_country["Link-team"]):
            URL = f'https://www.fifaindex.com{link_team}'
            r = requests.get(URL)
            page = r.content
            soup = BeautifulSoup(page, 'html5lib')
            li_class = soup.find_all('li',class_="list-group-item")

            rival_team = li_class[0].find("a", class_="link-team")
            Rival_team_list.append(rival_team.text)


# Fifa index does not show the budget for any team for the 2005 and 2006 years,
# so we will show the same budget for these years as the budget of 2007

# Let's also transform the budget in a number (millions of €)

            if ((Year == 2005) or (Year == 2006)):

                URL_original = f'https://www.fifaindex.com{link_team}'

                URL = URL_original[:-3] + '07/'

                r = requests.get(URL)
                page = r.content
                soup = BeautifulSoup(page, 'html5lib')
                li_class = soup.find_all('li',class_="list-group-item")

                Budget_original = li_class[4].find("span", 
            class_="data-currency data-currency-euro float-right").text
                Budget_digits = re.findall(r'\d',Budget_original)
                Budget = int(''.join(Budget_digits))/1000000

                Team_budget_list.append(Budget)


            else:    

                Budget_original = li_class[4].find("span",
            class_="data-currency data-currency-euro float-right").text
                Budget_digits = re.findall(r'\d',Budget_original)
                Budget = int(''.join(Budget_digits))/1000000

                Team_budget_list.append(Budget)


        table_fifa_country["Rival_team"] = Rival_team_list
        table_fifa_country["Budget_Mill_€"] = Team_budget_list

        # Concat the table created for one team with the empty dataframe
        table_fifa_all_countries = pd.concat(
                [table_fifa_all_countries,
                 table_fifa_country],
            ignore_index = True, axis = 0)

    # Concat the table created for one year with the empty dataframe
    table_fifa_all_years = pd.concat(
            [table_fifa_all_years,
             table_fifa_all_countries],
         ignore_index = True, axis = 0)


table_fifa_all_years

<timed exec>:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


CPU times: user 6min 7s, sys: 4.53 s, total: 6min 12s
Wall time: 7min 15s


,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85.0,82.0,77.0,81.0,/team/236/fc-porto/fifa05/,Benfica,16.0
1,2005,Portugal,Benfica,80.0,74.0,74.0,75.0,/team/234/benfica/fifa05/,Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63.0,70.0,65.0,68.0,/team/237/sporting-lisbon/fifa05/,Benfica,1.7
3,2005,Portugal,Belenenses,67.0,62.0,62.0,64.0,/team/1889/belenenses/fifa05/,Boavista,0.4
4,2005,Portugal,Nacional,69.0,59.0,61.0,63.0,/team/1891/nacional/fifa05/,Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...
2063,2022,France,FC Nantes,72.0,74.0,72.0,73.0,/team/71/fc-nantes/,Stade Rennais,7.5
2064,2022,France,Stade de Reims,71.0,72.0,75.0,73.0,/team/379/stade-de-reims/,ESTAC Troyes,8.5
2065,2022,France,ESTAC Troyes,71.0,73.0,71.0,72.0,/team/294/estac-troyes/,Stade de Reims,6.0
2066,2022,France,FC Lorient,73.0,72.0,70.0,71.0,/team/217/fc-lorient/,Stade Rennais,7.0


In [21]:
# Create an excel file with the table
table_fifa_all_years.to_excel(
    'Table_fifa_all_years_original.xlsx', index = False)

# Add missing teams information

In [22]:
table_fifa_all_years = pd.read_excel(
    'Table_fifa_all_years_original.xlsx')

In [23]:
%%time

# Check if the number of teams per year and league of Fifa Index site
# matches the number of teams in the results table and add the ones
# that could be missing. I've noticed that although Fifa index has a
# specific page per each team, country and year, sometimes not all of
# teams appear in the table I used to extract all the teams in the
# previous step. We need to identify those teams and add them to the
# final table

table_results_all_years = pd.read_excel('Table_results_all_years_edited.xlsx')


info_all_missing_teams = []

for Year in Years:
    for Country in Countries:
        
        # Create a list with all the teams for each country and year,
        # for both fifa table and results table
        
        df_teams_fifa = table_fifa_all_years[((
        table_fifa_all_years['Year'] == int(f'{Year}')) &
        (table_fifa_all_years['Country'] == f'{Country}'
    ))]
        list_teams_fifa = list(set(list(df_teams_fifa['Name'])))

        df_teams = table_results_all_years[(
        (table_results_all_years['Year'] == int(f'{Year}')) &
        (table_results_all_years['Country'] == f'{Country}') 
    )]
        list_results_teams = list(set(list(df_teams['Home_team'])+list(df_teams['Away_team'])))
        

        # Compare the number of teams in fifa and results lists, to see if there is
        # any team missing
        
        if (len(list_teams_fifa)-len(list_results_teams)) < 0:
            info_missing_teams = (Year,Country,
            f'Number of teams missing in FIFA table: {(len(list_results_teams)-len(list_teams_fifa))}')
            
            info_all_missing_teams.append(list(info_missing_teams))
            
        elif (len(list_teams_fifa)-len(list_results_teams)) > 0:
            info_missing_teams = (Year,Country,
            f'Number of teams missing in results table: {len(list_teams_fifa)-len(list_results_teams)}')
            
            info_all_missing_teams.append(list(info_missing_teams))


# Get the lists of teams from results and fifa tables, for each year and country
# with missing teams in the fifa table

for teams_missing in info_all_missing_teams:
    
    year_teams_missing = teams_missing[0]
    country_teams_missing = teams_missing[1]
    
    # Create an auxiliar dataframe (aux_table) to filter the table by the year and
    # country with teams missing
    aux_table = table_results_all_years[(table_results_all_years['Year']==int(year_teams_missing)) & 
                                   (table_results_all_years['Country']==country_teams_missing)]
    
    results_list = list(set(list(aux_table['Home_team'])+list(aux_table['Away_team'])))
    

    aux_table = table_fifa_all_years[(table_fifa_all_years['Year']==int(year_teams_missing)) & 
                                   (table_fifa_all_years['Country']==country_teams_missing)] 
                               
    fifa_list = list(set(aux_table['Name']))


    # Create another auxiliar dataframe (df1) to find a match between a results table
    # team name, and a fifa table name, applying the library fuzzywuzzy with a high
    # level of fuzzywuzzy score

    d = {'Name_from_results': results_list}
    df1 = pd.DataFrame(data=d)

    f = {'Name_from_fifa': fifa_list}
    df2 = pd.DataFrame(data=f) 
    
    # Get the suggested fifa team name for each of the team name in the results list
    df1['Name_from_fifa_suggested_by_fuzzywuzzy'] = df1['Name_from_results'].apply(
        (lambda x: process.extractOne(x, df2['Name_from_fifa'].to_list())[0]))
    
    # Get the score of the suggested fifa team name for each of the team name in the
    # results list
    df1['fuzzywuzzy_score'] = df1['Name_from_results'].apply(
        (lambda x: process.extractOne(x, df2['Name_from_fifa'].to_list())[1]))
    
    fuzzywuzzy_min_score = 90
    
    df1['Suggested_name_with_confidence'] = df1.apply(
        lambda x: x['Name_from_fifa_suggested_by_fuzzywuzzy'] 
        if x['fuzzywuzzy_score'] >= fuzzywuzzy_min_score
        else 'NONE', axis = 1)
    

    # Identify what teams do not have a high wuzzyfuzzy score

    list_results_teams_not_paired = []
    list_fifa_teams_not_paired = []
    
    for results_team in results_list:
        if results_team in list(df1[df1['Suggested_name_with_confidence']=='NONE']['Name_from_results']):
            list_results_teams_not_paired.append(results_team)
                                     
    for fifa_team in fifa_list:
        if fifa_team not in list(df1['Suggested_name_with_confidence']):
            list_fifa_teams_not_paired.append(fifa_team)


    # Find the best match between the teams of fifa that had not been assigned yet
    # to a results table team
    
    df1['Final_suggested_name'] = df1['Suggested_name_with_confidence']
    
    for fifa_team in list_fifa_teams_not_paired:
        results_name = process.extractOne(fifa_team, list_results_teams_not_paired)[0]
        
        # Get the index position of the results_name in the column "Name from results"
        index = df1[df1['Name_from_results'] == results_name].index[0]
        df1.at[index,'Final_suggested_name'] = fifa_team


    # Identify what results table teams name had been left alone (with no corresponding
    # fifa team)

    results_names_finally_missing = []
    
    for i in range(len(list(df1['Final_suggested_name']))):
        if df1['Final_suggested_name'][i] == 'NONE':
            results_names_finally_missing.append(df1['Name_from_results'][i])
    

    # Get the fifa name and fifa link team that corresponds to an results table team
    # left alone (for that we look to all the teams we see in fifa for all years)
    
    fifa_teams_list_year_country = list(set(
        list(table_fifa_all_years[
                                  table_fifa_all_years['Country']==country_teams_missing
                                 ]['Name']) +\
    list(table_fifa_all_years[ 
                                  table_fifa_all_years['Country']==country_teams_missing
                                 ]['Rival_team'])))
    
    
    for results_team in results_names_finally_missing:
        fifa_name = process.extractOne(results_team, fifa_teams_list_year_country)[0]

        Link_team = table_fifa_all_years.loc[table_fifa_all_years['Name'] == fifa_name,'Link-team'].iloc[0]
        Link_team = Link_team[:-3] + str(year_teams_missing)[-2:] + '/'

    # Now that we have the Fifa_link, we need to get the rest of the info relative
    # to the team, so we can add it to the fifa table

        URL = f'https://www.fifaindex.com{Link_team}'

        r = requests.get(URL)
        page = r.content
        soup = BeautifulSoup(page, 'html5lib')
        li_class = soup.find_all('li',class_="list-group-item")
        
        
        # Ratings
        
        for li_class_element in li_class:
            if li_class_element.text.startswith("Attack"):
                ATT = int(li_class_element.text[-2:])
            elif li_class_element.text.startswith("Midfield"):
                MID = int(li_class_element.text[-2:])
            elif li_class_element.text.startswith("Defence"):
                DEF = int(li_class_element.text[-2:])

        
        # We don't have the overall rating, so we can assume it as the average
        # of the 3 other ratings
        
        print('Year:',year_teams_missing)
        print('Country:',country_teams_missing)
        print('Team:',results_team)
        print('ATT:',ATT,' - MID:',MID,' - DEF:',DEF)
        print('\n')
            
        OVR = round(((ATT + MID + DEF)+0.0001)/3)
            
        
        # Rivals Teams

        rival_team = li_class[0].find("a", class_="link-team")
        Rival_team = rival_team.text

        
        # Budget
        
        if ((year_teams_missing == 2005) or (year_teams_missing == 2006)):

            URL = URL_original[:-3] + '07/'

            r = requests.get(URL)
            page = r.content
            soup = BeautifulSoup(page, 'html5lib')
            li_class = soup.find_all('li',class_="list-group-item")

            Budget_original = li_class[4].find("span", class_="data-currency data-currency-euro float-right").text
            Budget_digits = re.findall(r'\d',Budget_original)
            Budget = int(''.join(Budget_digits))/1000000

                    
        else:    

            Budget_original = li_class[4].find("span", class_="data-currency data-currency-euro float-right").text
            Budget_digits = re.findall(r'\d',Budget_original)
            Budget = int(''.join(Budget_digits))/1000000



        # Add the missing information collected to the fifa table

        new_row = {'Year':year_teams_missing,
                   'Country':country_teams_missing,
                   'Name': fifa_name,
                   'ATT':ATT,
                   'MID':MID,
                   'DEF':DEF,
                   'OVR':OVR,
                   'Link-team':Link_team,
                   'Rival_team':Rival_team,
                   'Budget_Mill_€':Budget
                  }

        table_fifa_all_years = table_fifa_all_years.append(new_row, ignore_index=True)
        

table_fifa_all_years

Year: 2012
Country: England
Team: Arsenal
ATT: 84  - MID: 80  - DEF: 81


Year: 2013
Country: England
Team: Southampton
ATT: 73  - MID: 74  - DEF: 71


Year: 2014
Country: Spain
Team: Osasuna
ATT: 71  - MID: 71  - DEF: 73


Year: 2014
Country: Spain
Team: Villarreal
ATT: 75  - MID: 74  - DEF: 72


CPU times: user 18.5 s, sys: 88.1 ms, total: 18.6 s
Wall time: 19.3 s


,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,Benfica,16.0
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,Benfica,1.7
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,Boavista,0.4
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...
2067,2022,France,Clermont Foot,72,71,72,71,/team/1815/clermont-foot/,AS Saint-Étienne,4.5
2068,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,Tottenham Hotspur,30.0
2069,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,Portsmouth,7.5
2070,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,Athletic Club de Bilbao,4.8


# Add columns Team-ID and Fifa_team_all_names

In [24]:
table_fifa_all_years 

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,Benfica,16.0
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,Benfica,1.7
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,Boavista,0.4
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...
2067,2022,France,Clermont Foot,72,71,72,71,/team/1815/clermont-foot/,AS Saint-Étienne,4.5
2068,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,Tottenham Hotspur,30.0
2069,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,Portsmouth,7.5
2070,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,Athletic Club de Bilbao,4.8


In [25]:
# Add 2 columns: one with the fifa index website team ID (unique per team),
# and a second column with all the different names for each single team
# present in the fifa index website (some teams have a different name in
# different years). These 2 columns will help a lot with merging the results
# table and fifa table, in the following notebook.

table_fifa_all_years['Team-ID'] = table_fifa_all_years['Link-team'].apply(
lambda x: x.split("/")[2]
)

list_all_names = []

for ID in list(table_fifa_all_years['Team-ID']):
    list_all_names.append(list(pd.unique(table_fifa_all_years[table_fifa_all_years['Team-ID']==ID]['Name'])))

table_fifa_all_years['Fifa_team_all_names'] = list_all_names


table_fifa_all_years = table_fifa_all_years[
    ['Year','Country','Name','ATT','MID','DEF','OVR','Link-team',
     'Team-ID','Fifa_team_all_names','Rival_team','Budget_Mill_€']
]

table_fifa_all_years

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
0,2005,Portugal,FC Porto,85,82,77,81,/team/236/fc-porto/fifa05/,236,"[FC Porto, F.C. Porto]",Benfica,16.0
1,2005,Portugal,Benfica,80,74,74,75,/team/234/benfica/fifa05/,234,"[Benfica, Sport Lisboa Benfica, SL Benfica]",Sporting Lisbon,5.0
2,2005,Portugal,Sporting Lisbon,63,70,65,68,/team/237/sporting-lisbon/fifa05/,237,"[Sporting Lisbon, Sporting CP Lisbon, Sporting...",Benfica,1.7
3,2005,Portugal,Belenenses,67,62,62,64,/team/1889/belenenses/fifa05/,1889,"[Belenenses, CF Os Belenenses, Belém, C.F. Os ...",Boavista,0.4
4,2005,Portugal,Nacional,69,59,61,63,/team/1891/nacional/fifa05/,1891,"[Nacional, Clube Desportivo Nacional, CD Nacio...",Marítimo,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
2067,2022,France,Clermont Foot,72,71,72,71,/team/1815/clermont-foot/,1815,[Clermont Foot],AS Saint-Étienne,4.5
2068,2012,England,Arsenal,84,80,81,82,/team/1/arsenal/fifa12/,1,"[Arsenal, Arsenal FC]",Tottenham Hotspur,30.0
2069,2013,England,Southampton,73,74,71,73,/team/17/southampton/fifa13/,17,[Southampton],Portsmouth,7.5
2070,2014,Spain,Osasuna,71,71,73,72,/team/479/osasuna/fifa14/,479,"[Osasuna, Club Atlético Osasuna, CA Osasuna]",Athletic Club de Bilbao,4.8


In [26]:
table_fifa_all_years[(table_fifa_all_years['Year'] == 2022) &\
                    (table_fifa_all_years['Country'] == 'Italy')]

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
2010,2022,Italy,Juventus,82,81,83,83,/team/45/juventus/,45,[Juventus],Inter,105.0
2011,2022,Italy,Inter,83,82,83,82,/team/44/inter/,44,"[Inter Milan, Inter]",Milan,99.5
2012,2022,Italy,Milan,81,79,79,81,/team/47/milan/,47,"[AC Milan, Milan]",Inter,37.0
2013,2022,Italy,Napoli,82,79,80,80,/team/48/napoli/,48,[Napoli],Juventus,56.0
2014,2022,Italy,Bergamo Calcio,81,79,79,80,/team/115845/bergamo-calcio/,115845,[Bergamo Calcio],Milan,20.0
2015,2022,Italy,Latium,79,80,79,79,/team/115841/latium/,115841,[Latium],Roma,39.5
2016,2022,Italy,Roma,79,79,78,79,/team/52/roma/,52,"[AS Roma, Roma]",Latium,46.0
2017,2022,Italy,Fiorentina,77,77,77,76,/team/110374/fiorentina/,110374,"[Firenze, Fiorentina, ACF Fiorentina]",Juventus,25.0
2018,2022,Italy,Bologna,78,74,73,75,/team/189/bologna/,189,[Bologna],Fiorentina,15.0
2019,2022,Italy,Hellas Verona,76,75,75,75,/team/206/hellas-verona/,206,[Hellas Verona],Napoli,8.0


In [27]:
# In 2022 there are 2 exceptions: fifa index website has given a new ID to
# 2 Italian teams: Lazio is called Latium and Atalanta is called Bergamo
# Calcio. It would be ok as later we use the library wuzzyfuzzy to match
# fifa index names with the results table, but as fifa index has also
# created a new ID for these 2 teams, we need to do a manual correction

table_fifa_all_years.loc[(
    table_fifa_all_years.Name == 'Lazio'), 'Fifa_team_all_names'] = '[Lazio, Latium]'

table_fifa_all_years.loc[(
    table_fifa_all_years.Name == 'Latium'), 'Fifa_team_all_names'] = '[Lazio, Latium]'

table_fifa_all_years.loc[(
    table_fifa_all_years.Name == 'Atalanta BC'), 'Fifa_team_all_names'] = '[Atalanta BC, Atalanta, Bergamo Calcio]'

table_fifa_all_years.loc[(
    table_fifa_all_years.Name == 'Atalanta'), 'Fifa_team_all_names'] = '[Atalanta BC, Atalanta, Bergamo Calcio]'

table_fifa_all_years.loc[(
    table_fifa_all_years.Name == 'Bergamo Calcio'), 'Fifa_team_all_names'] = '[Atalanta BC, Atalanta, Bergamo Calcio]'

In [28]:
table_fifa_all_years[(table_fifa_all_years['Year'] == 2022) &\
                    (table_fifa_all_years['Country'] == 'Italy')]

,Year,Country,Name,ATT,MID,DEF,OVR,Link-team,Team-ID,Fifa_team_all_names,Rival_team,Budget_Mill_€
2010,2022,Italy,Juventus,82,81,83,83,/team/45/juventus/,45,[Juventus],Inter,105.0
2011,2022,Italy,Inter,83,82,83,82,/team/44/inter/,44,"[Inter Milan, Inter]",Milan,99.5
2012,2022,Italy,Milan,81,79,79,81,/team/47/milan/,47,"[AC Milan, Milan]",Inter,37.0
2013,2022,Italy,Napoli,82,79,80,80,/team/48/napoli/,48,[Napoli],Juventus,56.0
2014,2022,Italy,Bergamo Calcio,81,79,79,80,/team/115845/bergamo-calcio/,115845,"[Atalanta BC, Atalanta, Bergamo Calcio]",Milan,20.0
2015,2022,Italy,Latium,79,80,79,79,/team/115841/latium/,115841,"[Lazio, Latium]",Roma,39.5
2016,2022,Italy,Roma,79,79,78,79,/team/52/roma/,52,"[AS Roma, Roma]",Latium,46.0
2017,2022,Italy,Fiorentina,77,77,77,76,/team/110374/fiorentina/,110374,"[Firenze, Fiorentina, ACF Fiorentina]",Juventus,25.0
2018,2022,Italy,Bologna,78,74,73,75,/team/189/bologna/,189,[Bologna],Fiorentina,15.0
2019,2022,Italy,Hellas Verona,76,75,75,75,/team/206/hellas-verona/,206,[Hellas Verona],Napoli,8.0


In [29]:
table_fifa_all_years.to_excel(
    'Table_fifa_all_years_edited.xlsx', index = False)